In [ ]:
! pip install telepot

In [ ]:
import json
import time
import requests
import telepot
from telepot.loop import MessageLoop
import paho.mqtt.client as mqtt
import os
import math

In [ ]:
class NotificationBot:
    def __init__(self, settings, chatID):
        self.settings = settings
        self.token = settings["telegramToken"]
        self.chatID = chatID

        # MQTT Setup
        self.mqtt_broker = os.getenv('MQTT_BROKER_HOST', 'mosquitto')
        self.mqtt_port = int(os.getenv('MQTT_BROKER_PORT', 1883))
        self.topic_observation = "building/observation"
        self.topic_action = "building/action"

        # Buffer to store the latest data received from MQTT
        self.last_observation = {}
        self.last_action = {}

        # Timestamp of the last sent alert
        self.last_alert_sent = 0

        # Initialize MQTT Client
        self.mqtt_client = mqtt.Client()
        self.mqtt_client.on_connect = self.on_mqtt_connect
        self.mqtt_client.on_message = self.on_mqtt_message
        self.mqtt_client.connect(self.mqtt_broker, self.mqtt_port)
        self.mqtt_client.loop_start()

        # Config mapping for zones
        self.zones = ["Tair_z1", "Tair_z2", "Tair_z4"]
        self.shades = {
            "Zone 1": ["Zona1_wall2_shade_FMU", "Zona1_wall8_shade_FMU", "Zona1_wall9_shade_FMU"],
            "Zone 2": ["Zona2_wall2_shade_FMU", "Zona2_wall3_shade_FMU"],
            "Zone 4": ["Zona4_wall2_shade_FMU"]
        }

        # Comfort thresholds
        self.t_min = 19
        self.t_max = 24

        self.bot = telepot.Bot(self.token)
        MessageLoop(self.bot, {'chat': self.on_chat_message}).run_as_thread()

    def on_mqtt_connect(self, client, userdata, flags, rc):
        print(f"✅ Bot connected to MQTT broker (rc={rc})")
        client.subscribe(self.topic_observation, qos=1)
        client.subscribe(self.topic_action, qos=1)

    def on_mqtt_message(self, client, userdata, msg):
        try:
            payload = json.loads(msg.payload)

            if msg.topic == self.topic_observation:
                # Logic for observations
                self.last_observation.update(payload)
                #print(f"[DEBUG] Received Observation: {payload}")
                # Check for comfort violations and alert if necessary
                self.monitor_and_alert(chat_id=self.chatID)
                
            elif msg.topic == self.topic_action:
                # Logic for actions
                self.last_action.update(payload)
                #print(f"[DEBUG] Received Action: {payload}")
                
            else:
                print(f"Unknown topic received: {msg.topic}")

        except Exception as e:
            print(f"Error parsing MQTT message: {e}")

    def on_chat_message(self, msg):
        content_type, chat_type, chat_id = telepot.glance(msg)
        text = msg.get('text', '').strip().lower()

        if text == '/temperature':
            self.send_temperature_status(chat_id)
        elif text == '/shades':
            self.send_shading_status(chat_id)
        elif text == '/help':
            msg = ("Commands:\n"
                "/temperature - Get temperature for all zones\n"
                "/shades - Get current blind positions\n")
            self.bot.sendMessage(chat_id, msg)

    def send_temperature_status(self, chat_id):
        """Fetch latest temperatures for all zones."""
        if not self.last_observation:
            self.bot.sendMessage(chat_id, "No data received yet via MQTT.")
            return

        status_msg = "🌡 **Current Temperatures:**\n"
        for zone in self.zones:
            temp = self.last_observation.get(zone)
            if temp is not None:
                truncated_temp = math.floor(float(temp) * 10) / 10 # Truncate to 1 decimal place without rounding
                status_msg += f"- {zone.replace('Tair_z', 'Zone ')}: {truncated_temp}°C\n"
            else:
                status_msg += f"- {zone.replace('Tair_z', 'Zone ')}: N/A\n"

        #print("Sending temperature status:", status_msg)
        self.bot.sendMessage(chat_id, status_msg, parse_mode='Markdown')

    def send_shading_status(self, chat_id):
        """Check if blinds are Open (0) or Closed (7)."""
        if not self.last_action:
            self.bot.sendMessage(chat_id, "No data received yet via MQTT.")
            return
        
        status_msg = "🪟 **Shading Status:**\n"
        for zone, shade_list in self.shades.items():
            status_msg += f"🏠 **{zone}**\n"
            for shade in shade_list:
                status_val = self.last_action.get(shade)
                # Displaying 7.0 as Closed and 0.0 as Open per FMU logic
                icon = "🌑" if status_val > 1.0 else "☀️"
                text = "Closed" if status_val > 1.0 else "Open"
                
                # Clean up field name for display (e.g., Zona1_wall2_shade_FMU -> Wall 2)
                parts = shade.split('_')
                label = parts[1].replace('wall', 'Wall ') 
                status_msg += f"  - {label}: {text} {icon}\n"
            status_msg += "\n"

        #print("Sending shading status:", status_msg)
        self.bot.sendMessage(chat_id, status_msg, parse_mode='Markdown')

    def monitor_and_alert(self, chat_id):
        """Check all zones for comfort violations (t_min-t_max°C)."""
        if not self.last_observation:
            return
        
        # Only alert if 1 min have passed since the last one
        current_time = time.time()
        if current_time - self.last_alert_sent < 60:
            return

        # Check for comfort violations
        uncomfort_zones = []
        for zone in self.zones:
            temp = self.last_observation.get(zone)

            if temp is not None:
                temp_float = float(temp)
                truncated_temp = math.floor(temp_float * 10) / 10 # Truncate to 1 decimal place without rounding

                if truncated_temp < self.t_min or truncated_temp > self.t_max:
                    uncomfort_zones.append((zone, truncated_temp))

        # Send alert if any zone is out of comfort range
        if uncomfort_zones:
            alert_msg = f"⚠️ **Comfort Alert!** The following zones are out of comfort range ({self.t_min}-{self.t_max}°C):\n"
            
            for zone_label, zone_temp in uncomfort_zones:
                alert_msg += f"- {zone_label.replace('Tair_z', 'Zone ')}: {zone_temp}°C\n"
                
            #print("Sending alert:", alert_msg)
            self.last_alert_sent = current_time
            self.bot.sendMessage(chat_id, alert_msg, parse_mode='Markdown')

In [ ]:
with open("settings.json") as f:
    conf = json.load(f)

TEST_CHAT_ID = 7201915868

bot = NotificationBot(conf, TEST_CHAT_ID)
print("Notification Bot started...")

try:
    while True:
        time.sleep(60)
except KeyboardInterrupt:
    print("Stopping Bot...")